In [40]:
! jupyter nbconvert --to python poc.ipynb

[NbConvertApp] Converting notebook poc.ipynb to python
[NbConvertApp] Writing 1866 bytes to poc.py


In [11]:
import pandas as pd
import requests
import plotly
import plotly.graph_objs as go

In [31]:
url = "https://pt.wikipedia.org/wiki/Predefinição:Tabela_Covid-19_Portugal_região/dia_(DGS)"

r = requests.get(url, auth=('user', 'pass'))
website = r.text
tables = pd.read_html( website, encoding="UTF-8")
table = tables[0]

In [32]:
table

data     Portugal                                                \
       data  confirmados                              recup.    óbitos   
       data            Δ            Σ            ‱    recup.    óbitos   
0    03 Mar            —            4         0000         0         0   
1    04 Mar            2            6         0001         0         0   
2    05 Mar            3            9         0001         0         0   
3    06 Mar            4           13         0001         0         0   
4    07 Mar            8           21         0002         0         0   
..      ...          ...          ...          ...       ...       ...   
208  27 Set          665        73604         7094     47647      1953   
209  totais        73604        73604         7094     47647      1953   
210    data  confirmados  confirmados  confirmados    recup.    óbitos   
211    data            Δ            Σ            ‱    recup.    óbitos   
212    data     Portugal     Portugal     Portugal  Portugal  Portugal   

           Norte                                    ...       Açores          \
     confirmados                            recup.  ...  confirmados  recup.   
               Δ            Σ            ‱  recup.  ...            ‱  recup.   
0              —            2         0001       —  ...         0000       —   
1              1            3         0001       —  ...         0000       —   
2              2            5         0001       —  ...         0000       —   
3              3            8         0002       —  ...         0000       —   
4              7           15         0004       —  ...         0000       —   
..           ...          ...          ...     ...  ...          ...     ...   
208          246        26407         7157       —  ...         1082       —   
209        26407        26407         7157       —  ...         1082       —   
210  confirmados  confirmados  confirmados  recup.  ...  confirmados  recup.   
211            Δ            Σ            ‱  recup.  ...            ‱  recup.   
212        Norte        Norte        Norte   Norte  ...       Açores  Açores   

                 Madeira                                              \
     óbitos  confirmados                             recup.   óbitos   
     óbitos            Δ            Σ            ‱   recup.   óbitos   
0         0            —            0         0000        —        0   
1         0            0            0         0000        —        0   
2         0            0            0         0000        —        0   
3         0            0            0         0000        —        0   
4         0            0            0         0000        —        0   
..      ...          ...          ...          ...      ...      ...   
208      15            3          217         0810        —        0   
209      15          217          217         0810        —        0   
210  óbitos  confirmados  confirmados  confirmados   recup.   óbitos   
211  óbitos            Δ            Σ            ‱   recup.   óbitos   
212  Açores      Madeira      Madeira      Madeira  Madeira  Madeira   

     estrangeiro    data  
     estrangeiro    data  
     estrangeiro  data.1  
0              0  03 Mar  
1              0  04 Mar  
2              0  05 Mar  
3              0  06 Mar  
4              0  07 Mar  
..           ...     ...  
208            —  27 Set  
209            —  totais  
210  estrangeiro    data  
211  estrangeiro    data  
212  estrangeiro    data  

[213 rows x 43 columns]

In [33]:
# select relevant columns and drop last helper rows
table = table[['data','Madeira']].drop(table.tail(4).index)
table 

data             Madeira                         
       data         confirmados            recup. óbitos
       data  data.1           Δ    Σ     ‱ recup. óbitos
0    03 Mar  03 Mar           —    0  0000      —      0
1    04 Mar  04 Mar           0    0  0000      —      0
2    05 Mar  05 Mar           0    0  0000      —      0
3    06 Mar  06 Mar           0    0  0000      —      0
4    07 Mar  07 Mar           0    0  0000      —      0
..      ...     ...         ...  ...   ...    ...    ...
204  23 Set  23 Set           5  210  0784      —      0
205  24 Set  24 Set           1  211  0788      —      0
206  25 Set  25 Set           0  211  0788      —      0
207  26 Set  26 Set           3  214  0799      —      0
208  27 Set  27 Set           3  217  0810      —      0

[209 rows x 7 columns]

In [34]:
# drop unneeded levels 
table.columns = table.columns.droplevel([0,1])
table

,data,data.1,Δ,Σ,‱,recup.,óbitos
0,03 Mar,03 Mar,—,0,0000,—,0
1,04 Mar,04 Mar,0,0,0000,—,0
2,05 Mar,05 Mar,0,0,0000,—,0
3,06 Mar,06 Mar,0,0,0000,—,0
4,07 Mar,07 Mar,0,0,0000,—,0
...,...,...,...,...,...,...,...
204,23 Set,23 Set,5,210,0784,—,0
205,24 Set,24 Set,1,211,0788,—,0
206,25 Set,25 Set,0,211,0788,—,0
207,26 Set,26 Set,3,214,0799,—,0


In [35]:
table = table.loc[:,['Σ', 'Δ']]

In [36]:
table

,Σ,Δ
0,0,—
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
204,210,5
205,211,1
206,211,0
207,214,3


In [37]:
# create new, proper date column, starting from March 1st; assuming update each day
start_date = '2020-03-03'
end_date = pd.to_datetime(start_date) + pd.DateOffset(days=len(table)-1)
dates = pd.date_range(start_date, end=end_date)

In [38]:
table['date'] = pd.Series(dates, index=table.index).values
table

,Σ,Δ,date
0,0,—,2020-03-03
1,0,0,2020-03-04
2,0,0,2020-03-05
3,0,0,2020-03-06
4,0,0,2020-03-07
...,...,...,...
204,210,5,2020-09-23
205,211,1,2020-09-24
206,211,0,2020-09-25
207,214,3,2020-09-26


In [39]:
trace = go.Scatter(x=table['date'], y=table['Σ'], name='sum of COVID cases')
data = [trace]

updatemenus = list([
    dict(active=1,
         buttons=list([
            dict(label='Log Scale',
                 method='update',
                 args=[{'visible': [True]},
                       {'title': 'Log scale',
                        'yaxis': {'type': 'log'}}]),
            dict(label='Linear Scale',
                 method='update',
                 args=[{'visible': [True]},
                       {'title': 'Linear scale',
                        'yaxis': {'type': 'linear'}}])
            ]),
        )
    ])

layout = dict(updatemenus=updatemenus, title='Linear scale')
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig)